In [2]:
import sqlite3
import csv
import pandas as pd
import os
import re

import math
from tqdm import tqdm
import multiprocessing as mp
import time


from difflib import SequenceMatcher as SeqMatcher
import numpy as np

folder_prefix = '/home/jovyan/work/'


In [3]:
new_data = pd.read_pickle(folder_prefix+ 'charters_full_2015.pkl')

In [4]:
# keywords = ['values', 'academics', 'skills', 'purpose',
#                        'direction', 'mission', 'vision', 'vision', 'mission', 'our purpose',
#                        'our ideals', 'ideals', 'our cause', 'curriculum','curricular',
#                        'method', 'pedagogy', 'pedagogical', 'approach', 'model', 'system',
#                        'structure','philosophy', 'philosophical', 'beliefs', 'believe',
#                        'principles', 'creed', 'credo', 'values','moral', 'history', 'our story',
#                        'the story', 'school story', 'background', 'founding', 'founded',
#                        'established','establishment', 'our school began', 'we began',
#                        'doors opened', 'school opened', 'about us', 'our school', 'who we are',
#                        'our identity', 'profile', 'highlights']

# mission_keywords = ['mission','vision', 'vision:', 'mission:', 'our purpose', 'our ideals', 'ideals:', 'our cause', 'cause:', 'goals', 'objective']
# curriculum_keywords = ['curriculum', 'curricular', 'program', 'method', 'pedagogy', 'pedagogical', 'approach', 'model', 'system', 'structure']
# philosophy_keywords = ['philosophy', 'philosophical', 'beliefs', 'believe', 'principles', 'creed', 'credo', 'value',  'moral']
# history_keywords = ['history', 'story','our story', 'the story', 'school story', 'background', 'founding', 'founded', 'established', 'establishment', 'our school began', 'we began', 'doors opened', 'school opened']
# about_keywords =  ['about us', 'our school', 'who we are', 'overview', 'general information', 'our identity', 'profile', 'highlights']

# mission_keywords = set(stemmer.stem(word) for word in mission_keywords)
# curriculum_keywords = set(stemmer.stem(word) for word in curriculum_keywords)
# philosophy_keywords = set(stemmer.stem(word) for word in philosophy_keywords)
# history_keywords = set(stemmer.stem(word) for word in history_keywords)
# about_keywords =  set(stemmer.stem(word) for word in about_keywords)


In [5]:
# m = '[(\'https://www.aaechighschools.com/\', \'False\', \'0\', "Home\\n | \\nParent / Student Login \\nCall Today!\\n602.297.8500\\nPrograms\\nAcademics\\nEquine Studies\\nMath & Sciences\\nVeterinary & Medical\\nOur Schools\\nSouth Mountain\\nCalendar\\nParadise Valley\\nCalendar\\nRed Mountain\\nCalendar\\nEstrella Mountain\\nCalendar\\nPrescott Valley\\nCalendar\\nMesa\\nCalendar\\nFor Parents\\nStudent Life\\nAbout AAEC\\nHistory & Philosophy\\nGoverning Board\\nCommunity Relations\\nRegistration\\nOther Scholarship Opportunities\\nResources\\nAccreditation\\nSPED\\nCareers\\nAAEC â\x80\x93 In the News\\nResource Guides\\nPolicies\\nFAQ\\nBlog\\nCommunity Service Events\\nAnnual Financial Reports\\nPre-Enroll\\nContact Us\\nBlog\\nMesa Campus Now Open!\\nAAEC is also building a brand new facility next to MCC in Mesa!!! Call today for details on how you can benefit from AAEC 480-222-3999.\\nJoin Today\\nSpring semester college classes have started for our High School students. Call today for details on how you can benefit from AAEC.\\nPrograms\\nStudents who enroll in our college prep programs can start earning college credits while in high school. \\nParents\\nAAEC is a leading college prep high school with locations in South Mountain, Red Mountain, Estrella Mountain, Paradise Valley and Prescott Valley.\\nSchools\\nArizona Agribusiness & Equine Center (AAEC) is a leading public charter school offering students the opportunity to earn college credits\\nAbout AAEC\\nAAEC high schools offer college preparatory curriculum, and enable students to earn college credits while completing work for their high school diploma.\\nRequest More Info\\n About Our\\n Programs!\\n\\t\\tName (*)\\nInvalid Input\\t\\t\\n\\t\\tEmail (*)\\nInvalid Input\\t\\t\\n\\t\\tPhone\\nInvalid Input\\t\\t\\n\\t\\tCampus\\nSouth Mountain\\nParadise Valley\\nRed Mountain\\nEstrella Mountain\\nPrescott Valley\\nMesa\\nInvalid Input\\t\\t\\nSouth Mountain\\nParadise Valley\\nRed Mountain\\nEstrella Mountain\\nPrescott Valley\\nMesa\\nAcademics\\nEquine Studies\\nMath & Sciences\\nVeterinary & Medical\\n\\r Public Early College High School\\nMission\\nAAEC Early College High School prepares young adults for success now and in the future by promoting lifelong learning through rigorous academic instruction, promoting social responsibility and employability, and providing motivated students with the opportunity to earn college credits while completing their high school requirements.\\nAAEC Early College High School vigorously participates in school-to-career initiatives and occupational education at state and national levels. Please visit the \\nArizona Department of Education\\n website to view AAEC\'s most recent school report card.\\nEnroll in our \\ncollege preparatory classes\\n and start earning college credits today. Contact AAEC Early College High School for information about our \\ncampuses\\n, read our \\nFAQ\\n, or \\nenroll online\\n today.\\nHighly-Qualified Educators Who Make a Difference â\x80¢ Rich Curriculum that Exceeds State Standards â\x80¢ FREE COLLEGE TUITION*\\r\\n*For Qualified Arizona Resident Students\\nPrograms\\nOur Schools\\nFor Parents\\nStudent Life\\nAbout AAEC\\nSouth Mountain\\nParadise Valley\\nRed Mountain\\nEstrella Mountain\\nPrescott Valley\\nResources\\nTax Credit Form\\nPre-Enroll\\nSitemap\\nCareers\\nArizona Agribusiness & Equine Center\\n District Office\\n 3636 N Central Ave Suite 1050\\n Phoenix, AZ 85012\\n ph. 602-297-8500\\n fx. 602-297-8540\\nGoverning Board Meeting Agenda\\nÂ© Copyright 2013 by Arizona Agribusiness & Equine Center. All Rights Reserved")'
# rm = 

In [6]:
new_data.columns

Index(['CMO_NAME', 'CMO_MEMSUM', 'SCH_NAME', 'CMO_STATE', 'CMO_SCHNUM',
       'CMO_URL', 'CMO_NUMSTATES', 'CMO_ALLSTATES', 'CMO_SECTOR',
       'CMO_NUMSTUDENTS_CREDO17',
       ...
       'FTE', 'YEAR_OPENED', 'YEAR_CLOSED', 'SEARCH16', 'TOTETH', 'PCTETH',
       'AGE', 'PCTFRL', 'PLACE', 'STR'],
      dtype='object', length=391)

In [7]:
# k = new_data.iloc[7:11]['CMO_WEBTEXT'][0]
# d = '(\'https://'
# s =  [d+e for e in k.split(d) if e]
# #k.split("(\'https://")
# k
# pr = ast.literal_eval(k)
# len(pr)

In [8]:
# #all_pages is a list of strings of all web page texts
# all_pages = []

# no_nan_data = new_data.loc[:, ['CMO_WEBTEXT']].fillna("")
# col_pages = no_nan_data.loc[1:3]['CMO_WEBTEXT'] #change to new_data['data'] later, but work with first 1 schools for now
# i = 0
# count = 0
# for school_data in col_pages:

#     for tup in school_data:
#         all_pages.append(tup[3])
      

#     i+=1

In [9]:
# unique_pages_set = set(all_pages)

# #set already get rid of duplicate strings
# ratio_df = pd.DataFrame(np.array(list(unique_pages_set)), columns=["Page"])
# ratio_df["Ratios"] = np.nan
# ratio_df["Similar"] = np.nan #similar will hold similar indexes, > 0.90 ratios

# ratio_df['Ratios'] = ratio_df['Ratios'].astype('object')
# ratio_df['Similar'] = ratio_df['Similar'].astype('object')

In [10]:
def create_sim_df(pages):
    
    ratio_df = pd.DataFrame(np.array(pages), columns=["Page"])
    ratio_df["Ratios"] = np.nan
    ratio_df["Similar"] = np.nan #similar will hold similar indexes, > 0.90 ratios

    ratio_df['Ratios'] = ratio_df['Ratios'].astype('object')
    ratio_df['Similar'] = ratio_df['Similar'].astype('object')
    
    
    
    if (len(pages) < 100) :
   
        for a in range(len(pages)):
            sim_ind_list = []
            for b in range(len(pages)):
                #add on every other index execpt the one it's on
                #this way, it'll compare every page to every other page for similarities
                if (a != b):
                    sim_ind_list.append(b)
            ratio_df['Similar'][a] = np.asarray(sim_ind_list)    
   
    else :
        index = 0
        for page0 in pages:
            ratios_list = []
            for page1 in pages:
                ratios_list.append(SeqMatcher(None, page0, page1).ratio())
            ratio_df['Ratios'][index] = ratios_list
            sim_ind_list = np.asarray(np.where((np.asarray(ratios_list) >= 0.7) & (np.asarray(ratios_list) != 1.0))[0]).tolist()
            ratio_df['Similar'][index] = sim_ind_list
            index+=1
                
    num_rows = ratio_df.shape[0] 
    tot_tuples = []

    final_cut_strings = [None] * num_rows
    #count_not_similar = 0  
    #not_sim_list = []
    sim_list = []

    for r in range(num_rows):
        if (len(ratio_df['Similar'][r]) != 0):
            sim_list.append(r)
            #print("has similar > 0.9 at index : " + str(r))
            list_cut = []
            for ind in ratio_df['Similar'][r]:
                list_of_triples = SeqMatcher(None, ratio_df['Page'][r], ratio_df['Page'][ind]).get_matching_blocks()
                zeroth_triple = list_of_triples[0] #first triple, most likely in beginning, most likely a header
                n = zeroth_triple[2] #j to j+n are the indices of the overlapping part
                orig_string = ratio_df['Page'][ind]
                cut_down_string = orig_string[:zeroth_triple[1]] + orig_string[zeroth_triple[1] + n:] #removes overlapping part
                list_cut.append([ind, cut_down_string])
            tot_tuples.extend(list_cut) #list of tuples
        else:
            #print("no similar at index : " + str(r))
            #not_sim_list.append(r)
            #count_not_similar +=1
            final_cut_strings[r]= ratio_df['Page'][r]
    
    return tot_tuples, final_cut_strings, ratio_df




In [11]:
# index = 0
# for page0 in unique_pages_set:
#     ratios_list = []
#     for page1 in unique_pages_set:
#         ratios_list.append(SeqMatcher(None, page0, page1).ratio())
#     ratio_df['Ratios'][index] = ratios_list
#     sim_ind_list = np.asarray(np.where((np.asarray(ratios_list) > 0.9) & (np.asarray(ratios_list) != 1.0))[0]).tolist()
#     ratio_df['Similar'][index] = sim_ind_list
#     index+=1

In [12]:
# num_rows = ratio_df.shape[0] 
# tot_tuples = []

# final_cut_strings = [None] * num_rows
# count_not_similar = 0  
# not_sim_list = []
# sim_list = []

# for r in range(num_rows):
#     if (len(ratio_df['Similar'][r]) != 0):
#         sim_list.append(r)
#         #print("has similar > 0.9 at index : " + str(r))
#         list_cut = []
#         for ind in ratio_df['Similar'][r]:
#             list_of_triples = SeqMatcher(None, ratio_df['Page'][r], ratio_df['Page'][ind]).get_matching_blocks()
#             zeroth_triple = list_of_triples[0] #first triple, most likely in beginning, most likely a header
#             n = zeroth_triple[2] #j to j+n are the indices of the overlapping part
#             orig_string = ratio_df['Page'][ind]
#             cut_down_string = orig_string[:zeroth_triple[1]] + orig_string[zeroth_triple[1] + n:] #removes overlapping part
#             list_cut.append([ind, cut_down_string])
#         tot_tuples.extend(list_cut) #list of tuples
#     else:
#         #print("no similar at index : " + str(r))
#         not_sim_list.append(r)
#         count_not_similar +=1
#         final_cut_strings[r]= ratio_df['Page'][r]

In [13]:
# list_grouped = [[] for x in range(num_rows)] #big list, just put list of strings in spots where needed
# list_indices_of_groups = []
# for ind in range(num_rows):
#     for tup in tot_tuples:
#         if(tup[0] == ind):
#             list_grouped[ind].append(tup[1]) #attach that tuple's string
#             list_indices_of_groups.append(tup[0])

In [14]:
# #not used
# unique_group_ind = set(list_indices_of_groups) 

In [15]:
# ind_fill_final_grouped = []
# i = 0
# for group in list_grouped:
#     if (len(group) != 0):
#         #print("list ready to insert at index " + str(i) + "\n")
#         ind_fill_final_grouped.append(i)
    
#     i+=1

In [16]:
# spot = 0
# #add into final cut strings, the new "cut down" versions of appropriate strings
# for li in list_grouped:
#     if (len(li) != 0):
#         #print(unique_ind_list[spot])
#         final_cut_strings[ind_fill_final_grouped[spot]]= min(li, key=len) #inserts into correct index, what was None before, add in that string now
#         spot+=1

In [17]:
def create_first_header_cut(tot_tuples, final_cut_strings, num_rows):
    #num_rows = ratio_df.shape[0]
    list_grouped = [[] for x in range(num_rows)] #big list, just put list of strings in spots where needed
    #list_indices_of_groups = []
    for ind in range(num_rows):
        for tup in tot_tuples:
            if(tup[0] == ind):
                list_grouped[ind].append(tup[1]) #attach that tuple's string
                #list_indices_of_groups.append(tup[0])
                
    ind_fill_final_grouped = []
    i = 0
    for group in list_grouped:
        if (len(group) != 0):
            ind_fill_final_grouped.append(i)
    
        i+=1
        
    spot = 0
    #add into final cut strings, the new "cut down" versions of appropriate strings
    for li in list_grouped:
        if (len(li) != 0):
            #print(unique_ind_list[spot])
            final_cut_strings[ind_fill_final_grouped[spot]]= min(li, key=len) #inserts into correct index, what was None before, add in that string now
            spot+=1
    
    return final_cut_strings


In [18]:
# #first removal of headers in final_cut_Strings currently, but now we want to cut down headers more
# #take out text before the first sentence or text before the first group of 7+ words

# super_final_strings = []
# for s in final_cut_strings:
#     use_punc = False
#     use_sev = False
#     punc = [",", ".", ":", ";"]
#     p_list = []
#     for p in punc:
#         if (s.find(p) != -1):
#             p_list.append(s.find(p))
#         else:
#             p_list.append(len(s))
    
#     punc_ind = min(p_list)
    
#     n_list = [index for index, k in enumerate(s) if k=='\n']
#     start_punc = len(s)
#     for i in n_list:
#         if(i < punc_ind):
#             start_punc = i # start_punc equals the largest index of \n that's less than index of first punctuation
    
#     start = 0
#     end = 0
#     total = ""
#     list_totals = []
#     st_en = []
#     for c in s:
#         if (c not in ['\n', '\t']):
#             total+=(c)
#             end+=1
#         else :
#             if(len(total.split()) >= 7): # we hit 7 words or more, wipe everything before start index
#                 #list_totals.append(total)
#                 st_en.append((start, end))
    
#             total= ""
#             start = end
#     start_sev = len(s)-1 #len(s)-1 #make it huge by default; if there's no group of 7, then start punc will be the smallest
#     if len(st_en) > 0:
#         start_sev = st_en[0][0] #index of first group of words that's >= 7 words; 0th tuple's start value
   

#     #take smaller of the two indices, since we want to use the property which occurs first
#     if start_punc < start_sev:
#         #if start of sentence which ends in/contains puncuation occurs earlier, wipe eveything before that index
#         #only take that index +1 and on, start right after the new line
#         new_string = s[start_punc+1:] 
#         super_final_strings.append(new_string)
        
#     else:
#         #if start of group of words that >= 7 occurs earlier than a sentence with punctuation, wipe eveything before that index
#         #only take that index and on, statr using that begining of the group of 7+ words
#         new_string = s[start_sev:] 
#         super_final_strings.append(new_string)
        
     
            

In [19]:
def create_second_header_cut(first_header_cut):
    
    super_final_strings = []
    for s in first_header_cut:
        s_new = ""
        if(s is None):
            s_new = ""
        else:
            s_new = s
        use_punc = False
        use_sev = False
        punc = [",", ".", ":", ";"]
        p_list = []
        for p in punc:
            if (s_new.find(p) != -1):
                p_list.append(s_new.find(p))
            else:
                p_list.append(len(s_new))

        punc_ind = min(p_list)

        n_list = [index for index, k in enumerate(s_new) if k=='\n']
        start_punc = len(s_new)
        for i in n_list:
            if(i < punc_ind):
                start_punc = i # start_punc equals the largest index of \n that's less than index of first punctuation

        start = 0
        end = 0
        total = ""
        list_totals = []
        st_en = []
        for c in s_new:
            if (c not in ['\n', '\t']):
                total+=(c)
                end+=1
            else :
                if(len(total.split()) >= 7): # we hit 7 words or more, wipe everything before start index
                    #list_totals.append(total)
                    st_en.append((start, end))

                total= ""
                start = end
        start_sev = len(s_new)-1 #len(s)-1 #make it huge by default; if there's no group of 7, then start punc will be the smallest
        if len(st_en) > 0:
            start_sev = st_en[0][0] #index of first group of words that's >= 7 words; 0th tuple's start value


        #take smaller of the two indices, since we want to use the property which occurs first
        if start_punc < start_sev:
            #if start of sentence which ends in/contains puncuation occurs earlier, wipe eveything before that index
            #only take that index +1 and on, start right after the new line
            new_string = s_new[start_punc+1:] 
            super_final_strings.append(new_string)

        else:
            #if start of group of words that >= 7 occurs earlier than a sentence with punctuation, wipe eveything before that index
            #only take that index and on, statr using that begining of the group of 7+ words
            new_string = s_new[start_sev:] 
            super_final_strings.append(new_string)
        
    return super_final_strings



In [20]:
#compare between pages of each school

def remove_string_overlaps(tuplist):
   
    unique_tuplist = []
    seen_pages = set() # Initialize list of known pages for a school
    unique_pages=[]
    reversed_pages = []
    tup_indices = []

    cleaned_strings = []
    new_list = []
    if str(tuplist) == 'nan':
        return new_list
    else :
        
        for tup in tuplist:
            if str(tup) != 'nan':
                seen_pages.add(tup[3])

        for i in range(len(tuplist)):
            if(str(tuplist[i]) != 'nan') and (tuplist[i][3] in seen_pages) and (tuplist[i][3]  not in unique_pages) and (tuplist[i][3] is not None):
                unique_tuplist.append(tuplist[i])
                unique_pages.append(tuplist[i][3])
                reversed_pages.append(tuplist[i][3][::-1])
                tup_indices.append(i)
                #print("unique page : " + str(i))

        #now compare all pages with each other 
        #print(unique_tuplist)
        tot_tuples, final_cut_strings, ratio_df = create_sim_df(unique_pages)
        first_header_cut = create_first_header_cut(tot_tuples, final_cut_strings, ratio_df.shape[0])

        #first removal of headers in final_cut_Strings currently, but now we want to cut down headers more
        #take out text before the first sentence or text before the first group of 7+ words  
        second_header_cut = create_second_header_cut(first_header_cut)


        #now run process on reversed strings

        rev_tot_tuples, rev_final_cut_strings, rev_ratio_df = create_sim_df(reversed_pages)

        rev_first_header_cut = create_first_header_cut(rev_tot_tuples, rev_final_cut_strings, rev_ratio_df.shape[0])

        for i in range(len(rev_first_header_cut)):
            #find where to cut the footer off , index

            add_string = second_header_cut[i]
            if i in tup_indices: 
                if(rev_first_header_cut is None):
                    rev_first_header_cut = ""
                if(rev_first_header_cut[i] is None):
                    rev_first_header_cut[i] = ""
                forward_string = rev_first_header_cut[i][::-1]
                #print(forward_string)
                sept = int(len(forward_string)/2)
                half_string = forward_string[len(forward_string) - sept:]
                #find that half in the regular string, and get the end of the half
                #print(type(second_header_cut[i]))
                #print(type(half_string))
                end_index = (second_header_cut[i]).find(half_string) + len(half_string) - 1
                add_string = second_header_cut[i][:end_index]
                #remove the footer aka remove stuff after the end_index
                #keep the stuff before end_index

            cleaned_strings.append(add_string)


        #then iterate through cleaned_strings and inset into each tuple

        for count in range(len(cleaned_strings)):
            new_tup = (tuplist[tup_indices[count]][0], tuplist[tup_indices[count]][1], tuplist[tup_indices[count]][2], cleaned_strings[count])
            new_list.append(new_tup)
    #         print(tup_indices[count])
    #         print(cleaned_strings[count])
    #         print("\n  \n")

    #     print(len(new_list))
    #     print(len(tup_indices))
        return new_list


#first make a list of tuples that has no nan values and has
#if not np.isnan(tuplist):   CHECK FOR NAN OUTSIDE

#k = remove_string_overlaps(new_data['WEBTEXT'][11])





In [23]:
index = 0
ki = []
lengths = []
new_data['WEBTEXT'] = new_data['WEBTEXT'].fillna("")
for li in new_data['WEBTEXT']:
    if len(li) <100:
        ki.append(index)
        
        #print(index)
    lengths.append(len(li))
    index+=1
print(len(ki))
print(len(new_data['WEBTEXT']))
print(np.average(lengths))
print(np.median(lengths))
ki

9757
11521
61.54882388681538
3.0


[0,
 1,
 2,
 3,
 4,
 5,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 18,
 19,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185,
 186,
 187,
 188

In [26]:
lengths

[5,
 5,
 5,
 5,
 1,
 1,
 178,
 184,
 214,
 49,
 28,
 49,
 49,
 25,
 41,
 40,
 41,
 219,
 1,
 3,
 361,
 3,
 1,
 1,
 61,
 61,
 1,
 0,
 10,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 48,
 48,
 1,
 1,
 1,
 1,
 1,
 1,
 90,
 90,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 36,
 36,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 44,
 44,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 42,
 1,
 1,
 1,
 1,
 3,
 3,
 3,
 2,
 2,
 6,
 9,
 9,
 9,
 9,
 2,
 12,
 12,
 11,
 11,
 2,
 2,
 12,
 13,
 1,
 1,
 1,
 2,
 1,
 1,


In [29]:
print(set(lengths))

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222,

In [ ]:
k = remove_string_overlaps(new_data['WEBTEXT'][6])

In [37]:
k

[('https://www.aaechighschools.com/public-charter-school/estrella-mountain.html',
  'False',
  '0',
  'on\n3400 N. Dysart Rd., Avondale, AZ 85392 (Dysart Rd. and Osborn Rd.) | Phone: 623.535.0754 | \nClick here for directions.\nWelcome from the Principal\nDear Students:\nOn behalf of all of us, “Welcome to AAEC - Estrella Mountain High School”!\xa0\nStudents such as you advance to AAEC for a variety of reasons. \xa0Some of you prefer AAEC because of the rigorous high school coursework offered coupled with the early college component that provides you the opportunity to take free classes at Estrella Mountain Community College. \xa0Some of you selected this campus because of its small size, the safe environment and family-like atmosphere while others selected Estrella Mountain as their school of choice and opted for the opportunity to earn college credit while concurrently earning high school credit applicable to high school graduation requirements. Regardless of the reason you personall

In [31]:
r = sorted(lengths, key=int)  
np.median(r)
for i in r:
    print(i)

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
14
14
14
14
14
14


77
77
77
77
77
77
77
77
77
77
77
77
77
77
77
77
77
77
77
77
77
78
78
78
78
78
78
78
78
78
78
78
78
78
78
78
78
78
78
78
78
78
78
78
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
81
81
81
81
81
81
81
81
81
81
81
81
81
81
81
81
81
81
81
81
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
83
83
83
83
83
83
83
83
83
83
83
83
83
83
83
83
83
83
83
83
83
83
83
83
83
83
83
83
83
83
83
83
83
83
84
84
84
84
84
84
84
84
84
84
84
84
84
84
84
84
84
84
84
84
84
84
84
84
84
84
84
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
86
86
86
86
86
86
86
86
86
86
86
86
86
86
86
86
86
86
86
86
86
86
86
87
87
87
87
87
87
87
87
87
87
87
87
87
87
87
87
87
87
87
87
87
87
87
87
87
87
87
87
87
87
87
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
89
89
89
89
89
89
89
89
89
89
89
89
89
89
89
89
89
89
89
89
89
89
89
89
89
89
89
90
90
90
90
90
90
90
90
90
90
90
90
90
90
9

In [20]:
#apply remove_string_overlaps on each school, aka on each row of new_data
#since pages of a school will likely be similar to the other pages within that school own

def parse_df(old_list):
    #print(loc)
    new_list = remove_string_overlaps(old_list)
    return new_list
    


In [ ]:
new_data['WEBTEXT'] = new_data['WEBTEXT'].fillna("")
arr_of_dfs = np.array_split(new_data, 44)

global merged_df_file
merged_df_file = folder_prefix+"merged_df_WEBTEXT.csv" # Prepare file name

In [ ]:
# tqdm.pandas(desc="Processing:")

# arr_of_dfs[0]['WEBTEXT'] = arr_of_dfs[0]['WEBTEXT'].progress_apply(parse_df)

# arr_of_dfs[0].to_csv(merged_df_file, mode="w", index=False, header=arr_of_dfs[0].columns.values, sep="\t", encoding="utf-8")


#df = pd.read_csv(folder_prefix+'_mergedf_WEBTEXT.csv', header=arr_of_dfs[0].columns.values, sep="\t", encoding="utf-8")

In [ ]:

global num

def chunk_assign(df_chunk):
    tqdm.pandas(desc="Processing inside a df chunk:")
    df_chunk['WEBTEXT'] = df_chunk['WEBTEXT'].progress_apply(parse_df)
    
    if num==0: # Save first slice to new file (overwriting if needed)
        df_chunk.to_csv(merged_df_file, mode="w", index=False, header=df_chunk.columns.values, sep="\t", encoding="utf-8")
    else:
        df_chunk.to_csv(merged_df_file, mode="a", index=False, header=False, sep="\t", encoding="utf-8")
        num+=1
    
    #free chunk?
    logging.info("df chunk saved to " + df_filepath )
    
    return df_chunk

In [ ]:
start_time = time.time()
tqdm.pandas(desc="Processing:")

p = mp.Pool(processes = 44)
result_df = p.map(chunk_assign, arr_of_dfs)

p.close()

print("--- %s seconds ---" % (time.time() - start_time))

In [113]:
# #list of common words in footers
# footer_list = ["Copyright", "All Rights Reserved",  "Read More", 
#                "Useful Links", "Search", "Survey", "Feed", "Fax", "Address",  "Sitemap", 
#               "Jobs"]
# #facebook, contact us, enroll etc occurs in headers as well, so had to take that out

# footers_removed_strings = []

# #look for the keyword in each string and if found, remove all the text after it
# for s in super_final_strings:
#     no_newline = s.replace("\n", " ")
#     new_list = []
#     for word in footer_list:
#         if (no_newline.find(word) != -1):
#             new_list.append(no_newline.find(word))
#         else :
#             new_list.append(len(s))

#     #get the index of the earliest occurence of a keyword
#     f_ind = min(new_list)
    
#     #go back to new line or period right before and wipe out everything after that
#     n_list = [index for index, k in enumerate(s) if k in ['\n', '.']]
#     start_punc = len(s)
#     for i in n_list:
#         if(i < f_ind):
#             start_punc = i # start_punc equals the largest index of \n or . that's less than index of the keyword
            
#     if start_punc < f_ind:
#             footer_rem = s[:start_punc]
#             footers_removed_strings.append(footer_rem) 
#     else:
#             footer_rem = s[:f_ind]
#             footers_removed_strings.append(footer_rem) 
              

In [114]:
# kr = super_final_strings[4]
# no_newline = kr.replace("\n", " ")
# new_list = []
# li = []
# for word in footer_list:
    
#     if (no_newline.find(word) != -1):
#         new_list.append(no_newline.find(word)) #index at which the word starts
#     else :
#         new_list.append(len(kr))

# #get the index of the earliest occurence of a keyword
# f_ind = min(new_list)
    
# #go back to new line or period right before and wipe out everything after that
# n_list = [index for index, k in enumerate(kr) if k in ['\n', '.']]
# start_punc = len(kr)
# for i in n_list:
#     if(i < f_ind):
#         start_punc = i # start_punc equals the largest index of \n or . that's less than index of the keyword
            
# if start_punc < f_ind: #if punctuation occurs before keyword, wipe out everything after punctuation
#         footer_rem = kr[:start_punc]
#         li.append(footer_rem) 
# else:
#         footer_rem = kr[:f_ind] #else just wipe out everything after punctuation
#         li.append(footer_rem) 
# print(word + ": " + "start_punc : " + str(start_punc) + " . f_ind : " + str(f_ind))

# li[0]


In [115]:
#kr[646:]

In [116]:
#footers_removed_strings are the final, most cut down versions of the web text
#we did process on "WEBTEXT" column of new_data
#now repeat for "CMO_WEBTEXT" column of new_data

In [120]:
# footers_removed_strings

['602.297.8500\nPrograms\nAcademics\nEquine Studies\nMath & Sciences\nVeterinary & Medical\nOur Schools\nSouth Mountain\nCalendar\nParadise Valley\nCalendar\nRed Mountain\nCalendar\nEstrella Mountain\nCalendar\nPrescott Valley\nCalendar\nMesa\nCalendar\nFor Parents\nStudent Life\nAbout AAEC\nHistory & Philosophy\nGoverning Board\nCommunity Relations\nRegistration\nOther Scholarship Opportunities\nResources\nAccreditation\nSPED\nCareers\nAAEC â\x80\x93 In the News\nResource Guides\nPolicies\nFAQ\nBlog\nCommunity Service Events\nAnnual Financial Reports\nPre-Enroll\nContact Us\nBlog\nCareers\nReview current career opportunities with AAEC.\nSouth Mountain\nParadise Valley\nRed Mountain\nEstrella Mountain\nPrescott Valley\nMesa\nAcademics\nEquine Studies\nMath & Sciences\nVeterinary & Medical\nCareer Center - AAEC High Schools\nArizona Agribusiness & Equine Center INC. (AAEC)\nAAEC was founded by Linda Proctor Downing, through the Arizona State Board for Charter Schools in 1997. AAEC belie

In [127]:
# for count in range(len(footers_removed_strings)):
#     if "\nPre-enroll" in footers_removed_strings[len(footers_removed_strings[count])-13:]:
#         print(str(count))
        

In [102]:
#ratio_df['Page'][4]

'Home\n | \nParent / Student Login \nCall Today!\n602.297.8500\nPrograms\nAcademics\nEquine Studies\nMath & Sciences\nVeterinary & Medical\nOur Schools\nSouth Mountain\nCalendar\nParadise Valley\nCalendar\nRed Mountain\nCalendar\nEstrella Mountain\nCalendar\nPrescott Valley\nCalendar\nMesa\nCalendar\nFor Parents\nStudent Life\nAbout AAEC\nHistory & Philosophy\nGoverning Board\nCommunity Relations\nRegistration\nOther Scholarship Opportunities\nResources\nAccreditation\nSPED\nCareers\nAAEC â\x80\x93 In the News\nResource Guides\nPolicies\nFAQ\nBlog\nCommunity Service Events\nAnnual Financial Reports\nPre-Enroll\nContact Us\nBlog\nPolicies\nRequest More Info\n About Our\n Programs!\n\t\tName (*)\nInvalid Input\t\t\n\t\tEmail (*)\nInvalid Input\t\t\n\t\tPhone\nInvalid Input\t\t\n\t\tCampus\nSouth Mountain\nParadise Valley\nRed Mountain\nEstrella Mountain\nPrescott Valley\nMesa\nInvalid Input\t\t\nSouth Mountain\nParadise Valley\nRed Mountain\nEstrella Mountain\nPrescott Valley\nMesa\nAcad

In [49]:
#footers_removed_strings[1]

' posts'

In [50]:
#super_final_strings[1]

' posts\nInterested in Big Data? Statistics? Chance? Apply For a â\x80\x9cNo Essayâ\x80\x9d Scholarship!\nPosted on \nMay 30, 2017\nby \nFWadmin\nPlanning for college and beyond requires you to be a student of money, big data and statisticsâ\x80\x94while also being a student at AAEC. You can use all of these subjects in the real world with a big data company based â\x80¦ \nContinue reading \nâ\x86\x92\nPosted in \nUncategorized\n|\t\t\t\t\t\t\t\t\t\t\t\t\nLeave a comment\nTree Rings: The New Kind of Record on a Turntable\nPosted on \nMay 18, 2017\nby \nFWadmin\nA DJ or Disc Jockey makes his or her living with vinyl records, turntables, speakers and a creative playlist. But hereâ\x80\x99s what happens when the DJ is a media artist and the LP is a disc made of wood: http://www.cpr.org/classical/blog/turntable-plays-tree-rings-instead-vinyl-grooves-how-did-he-do â\x80¦ \nContinue reading \nâ\x86\x92\nPosted in \nUncategorized\n|\t\t\t\t\t\t\t\t\t\t\t\t\nLeave a comment\nIn From Africa: Wh